## Problem Set 2: Merging and regular expressions

**Total points (without extra credit)**: 30 

**Background on the policy context**: here, we're going to use two datasets to practice reshaping, merging, and regular expression patterns. Both datasets relate to the broader issue of which employers might be violating the rights of temporary guestworkers granted visas under the H-2A program. Here are some articles about potential exploitation of guestworkers by firms and inequality caused by minimal oversight:

- News media coverage of labor abuses of temporary guestworkers: https://www.buzzfeednews.com/article/kenbensinger/the-pushovers 
- GAO report on labor abuses of temporary guestworkers: https://www.gao.gov/products/gao-15-154

The following datasets are located in `pset2_inputdata` (need to unzip): 

- `jobs_clean`: a dataset of guestworker jobs posted by many employers, some of whom have been debarred (banned) from the program for labor abuses; others not debarred
- `debar`: a dataset of employers who committed violations of labor regulations meant to protect temporary guestworkers 


You can view a codebook here: https://docs.google.com/spreadsheets/d/1rF9GJEC8pPKxipD0TsoG9DVdqz3EJ-b-BHEtyioAX7I/edit?usp=sharing


In [4]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import os

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# 1. Reshaping data (13 points total)

Load the following dataset stored in `pset2_inputdata`: `debar.csv`

This represents employers temporarily banned from hiring workers (debar.csv); call this `debar`


View the head()


In [5]:
debar = pd.read_csv("pset2_inputdata/debar.csv")
debar.head()

,Name,"City, State",Violation,Duration,Start date,End date
0,J&J Harvesting,"Leads, ND",Failure to respond to audit (partial response),2 years,1/19/2014,1/18/2016
1,"Stahlman Apiaries, Inc","Selby, SD",Failure to respond to audit (partial response),1 year,2/19/2015,2/14/2016
2,Trust Nursery,"Pulaski, NY",Failure to respond to audit (partial response),1 year,3/21/2014,3/20/2015
3,Anton Fertilizer Inc.,"Dighton, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016
4,"Great Plains Fluid Service, Inc.","Greensburg, KS",Failure to respond to audit (no response),2 years,3/30/2014,3/29/2016


## 1.1 (1 point)

Print the number of rows in `debar` versus the number of unique employer names (`Name`). Is there one row per employer or multiple rows for some employers?

In [6]:
print("The number of rows in debar:", len(debar))
print("The number of unique employer names:", debar['Name'].nunique())

The number of rows in debar: 114
The number of unique employer names: 98


Since the number of rows (114) is greater than the number of unique employer names (98), it indicates that there are multiple rows for some employers in the dataset. 

## 1.2 Investigating duplicated rows (2 points)

A. Create a new column in `debar`--`is_repeated`-- that tells us whether an employer (`Name`) is repeated > 1 times

*Hint*: there are multiple ways to solve this but some possibilities to get the list of names that are repeated are:
- Using value_counts() on the `Name` variable and extracting the index from that value counts 
- Using groupby to count the rows attached to one name

B. Print the rows where `is_repeated == True` and interpret

C. Subset to the rows where `is_repeated == True` and save that data as `mult_debar`. Print the head() and shape

In [7]:
# Get the counts of each employer name
name_counts = debar['Name'].value_counts()

# Create a new column 'is_repeated' indicating whether the name is repeated
debar['is_repeated'] = debar['Name'].apply(lambda x: name_counts[x] > 1)

In [8]:
print(debar[debar['is_repeated']].sort_values(by='Name'))

                                        Name       City, State  \
6                    Annabella Land & Cattle     Annabella, UT   
30                   Annabella Land & Cattle   Annabella, Utah   
29                       Autumn Hill Orchard        Groton, MA   
7                        Autumn Hill Orchard        Groton, MA   
8      Caddo Creek Ranch, dba Paradise Ranch         Caddo, TX   
28     Caddo Creek Ranch, dba Paradise Ranch      Caddo, Texas   
56                        Cisco Produce Inc.         Cairo, GA   
19                        Cisco Produce Inc.         Cairo, GA   
103                         Dove Creek Farms  Mount Vernon, TX   
109                         Dove Creek Farms  Mount Vernon, TX   
58                                 F&W Farms       Ingalls, KS   
18                                 F&W Farms       Ingalls, KS   
11                     Loewen Harvesting LLC   Brownsville, TX   
25                     Loewen Harvesting LLC    Brownfield, TX   
55        

There are slight differences in each repeated entry. For some employers, the state abbreviation is incorrect. For others, the start and/or end date is off by one day. However, for most employers, the "Violation" variable is not streamlined, so some violations have an extra dash or are worded slightly differently. For the most part, it seems that all of the data is the same for both entries, so it seems that these employers just happen to have duplicate entries.

In [9]:
mult_debar = debar[debar['is_repeated']]
mult_debar.head()
print("Shape:", mult_debar.shape)

,Name,"City, State",Violation,Duration,Start date,End date,is_repeated
6,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,True
7,Autumn Hill Orchard,"Groton, MA",Failure to respond to audit (no response),2 years,7/6/2014,7/5/2016,True
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,True
11,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,True
12,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True


Shape: (32, 7)


## 1.3 Reshape mult_debar to wide to begin filtering out duplicates (4 points)

You want to separate out two cases:

- Cases where the repeat rows for one employer are due to duplicated data 
- Cases where the repeat rows for one employer represent repeated violations for different issues

There are various ways to check duplicates in this data (eg converting `Violation` to lowercase; replacing spelled-out states with two-dig state codes)

We're going to use the simple rule of:

- A row is a duplicate if, within an employer (defined by Name + City, State), the Start date for each row's violation is the same 

To begin to check this, reshape `mult_debar` to a wide dataframe (`mult_debar_wide`) with the following columns, treating the `Name` and `City, State` as the index for the pivot:

- Name
- City, State
- start_date_viol1
- start_date_viol2

Print the head and shape

In [10]:
# Reshape mult_debar to a wide DataFrame (mult_debar_wide)
mult_debar_wide = mult_debar.pivot_table(index=['Name', 'City, State'],
                                         columns=mult_debar.groupby(['Name', 'City, State']).cumcount() + 1,
                                         values='Start date',
                                         aggfunc='first')

# Rename the columns
mult_debar_wide.columns = [f'start_date_viol{i}' for i in range(1, len(mult_debar_wide.columns) + 1)]

mult_debar_wide
print("Shape:", mult_debar_wide.shape)


start_date_viol1  \
Name                                    City, State                         
Annabella Land & Cattle                 Annabella, UT            5/9/2014   
                                        Annabella, Utah          5/9/2014   
Autumn Hill Orchard                     Groton, MA               7/6/2014   
Caddo Creek Ranch, dba Paradise Ranch   Caddo, TX               7/20/2014   
                                        Caddo, Texas            7/20/2014   
Cisco Produce Inc.                      Cairo, GA              12/10/2014   
Dove Creek Farms                        Mount Vernon, TX         2/9/2018   
F&W Farms                               Ingalls, KS            12/10/2014   
Loewen Harvesting LLC                   Brownfield, TX          8/20/2014   
                                        Brownsville, TX         8/20/2014   
Macky and Brad Farms                    Plains, TX              2/13/2015   
Maple Ridge Custom Services, LLC        Altheimer, AK          11/16/2014   
                                        Altheimer, AR          11/16/2014   
Mark Duncan                             Roosevelt, UT          11/16/2014   
Old Tree Farms/Verpaalen Custom Service Volga, SD              12/11/2014   
Rollo Farm Labor Contractor             Miami, FL               8/23/2014   
SRT Farms                               Morton, TX             11/16/2014   
Sharon Mathis                           Tifton, GA             11/16/2014   
Turner Farms                            Healy, KS                 7/17/19   
Xavier Horne                            Lyons, Georgia          6/16/2016   

                                                         start_date_viol2  
Name                                    City, State                        
Annabella Land & Cattle                 Annabella, UT                 NaN  
                                        Annabella, Utah               NaN  
Autumn Hill Orchard                     Groton, MA               7/6/2014  
Caddo Creek Ranch, dba Paradise Ranch   Caddo, TX                     NaN  
                                        Caddo, Texas                  NaN  
Cisco Produce Inc.                      Cairo, GA              12/10/2015  
Dove Creek Farms                        Mount Vernon, TX         2/9/2018  
F&W Farms                               Ingalls, KS            12/10/2014  
Loewen Harvesting LLC                   Brownfield, TX                NaN  
                                        Brownsville, TX               NaN  
Macky and Brad Farms                    Plains, TX              2/13/2015  
Maple Ridge Custom Services, LLC        Altheimer, AK                 NaN  
                                        Altheimer, AR                 NaN  
Mark Duncan                             Roosevelt, UT          11/16/2014  
Old Tree Farms/Verpaalen Custom Service Volga, SD               12/1/2014  
Rollo Farm Labor Contractor             Miami, FL               8/23/2014  
SRT Farms                               Morton, TX             11/16/2014  
Sharon Mathis                           Tifton, GA             11/16/2014  
Turner Farms                            Healy, KS                 7/17/19  
Xavier Horne                            Lyons, Georgia          9/27/2017

Shape: (20, 2)


In [31]:
# Count violations by employer
mult_debar['violation_count'] = mult_debar.groupby(['Name', 'City, State']).cumcount() + 1

# Reshape to wide DataFrame
mult_debar_wide = mult_debar.pivot_table(index=['Name', 'City, State'], 
                                         columns='violation_count', 
                                         values='Start date', 
                                         aggfunc='first').reset_index()

# Rename columns
mult_debar_wide.columns = ['Name', 'City, State', 'start_date_viol1', 'start_date_viol2']

# Print the head and shape
print(mult_debar_wide.head())
print("Shape:", mult_debar_wide.shape)


                                    Name      City, State start_date_viol1  \
0                Annabella Land & Cattle    Annabella, UT         5/9/2014   
1                Annabella Land & Cattle  Annabella, Utah         5/9/2014   
2                    Autumn Hill Orchard       Groton, MA         7/6/2014   
3  Caddo Creek Ranch, dba Paradise Ranch        Caddo, TX        7/20/2014   
4  Caddo Creek Ranch, dba Paradise Ranch     Caddo, Texas        7/20/2014   

  start_date_viol2  
0              NaN  
1              NaN  
2         7/6/2014  
3              NaN  
4              NaN  
Shape: (20, 4)


/var/folders/0h/crf07_0d3vg01fcwqpddyq180000gn/T/ipykernel_97494/2006067237.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mult_debar['violation_count'] = mult_debar.groupby(['Name', 'City, State']).cumcount() + 1


## 1.4 Filter out duplicates from original debar data (6 points)

A. Using `mult_debar_wide`, add a column `is_dup` that takes value of True for cases where start_date_viol1 == start_date_viol2 marking the row as a duplicate

B. Going back to the original long-format data you loaded at the beginning- `debar`
    - For employers where `is_dup == True` as indicated by your wide-format dataframe, only keep `violnum == viol1`
    - For all other employers (so is_dup == False and ones we didnt need to check duplicates for), keep all violnum
    - Remove the `is_repeated` column from the `debar` data

**Hint**: you can complete part B without a for loop; `pd.concat` with axis = 0 (row binding) is one way

Call the resulting dataframe `debar_clean` and print the shape and # of unique employer names

In [11]:
# Check if start_date_viol1 is equal to start_date_viol2, marking the row as a duplicate
mult_debar_wide['is_dup'] = mult_debar_wide['start_date_viol1'] == mult_debar_wide['start_date_viol2']

mult_debar_wide.head()


start_date_viol1  \
Name                                  City, State                        
Annabella Land & Cattle               Annabella, UT           5/9/2014   
                                      Annabella, Utah         5/9/2014   
Autumn Hill Orchard                   Groton, MA              7/6/2014   
Caddo Creek Ranch, dba Paradise Ranch Caddo, TX              7/20/2014   
                                      Caddo, Texas           7/20/2014   

                                                      start_date_viol2  is_dup  
Name                                  City, State                               
Annabella Land & Cattle               Annabella, UT                NaN   False  
                                      Annabella, Utah              NaN   False  
Autumn Hill Orchard                   Groton, MA              7/6/2014    True  
Caddo Creek Ranch, dba Paradise Ranch Caddo, TX                    NaN   False  
                                      Caddo, Texas                 NaN   False

In [12]:
# Reset index to bring 'Name' and 'City, State' back as columns
mult_debar_wide.reset_index(inplace=True)

# B. Merge debar with mult_debar_wide to add 'is_dup' information to each row in debar
debar_with_dup_info = pd.merge(debar, mult_debar_wide[['Name', 'City, State', 'is_dup']], on=['Name', 'City, State'], how='left')

# B. Filter out duplicates from the original debar data based on 'is_dup'
# For employers where is_dup == True, keep only the rows where start_date_viol1 == start_date_viol2
# For all other employers, keep all rows
debar_clean = debar_with_dup_info[(debar_with_dup_info['is_dup'] == True) & (debar_with_dup_info['Start date'] == debar_with_dup_info['Start date'].shift(-1))]
debar_clean = pd.concat([debar_clean, debar_with_dup_info[debar_with_dup_info['is_dup'] == False]], axis=0)

# Reset the index
debar_clean.reset_index(drop=True, inplace=True)

# Remove the 'is_repeated' column from debar_clean
debar_clean.drop(columns=['is_repeated'], inplace=True)

# Print the shape and number of unique employer names
print("Shape:", debar_clean.shape)
print("Number of unique employer names:", debar_clean['Name'].nunique())

Shape: (21, 7)
Number of unique employer names: 12


In [13]:
debar_clean

,Name,"City, State",Violation,Duration,Start date,End date,is_dup
0,Rollo Farm Labor Contractor,"Miami, FL",Failure to respond to audit (no response),2 years,8/23/2014,8/22/2016,True
1,Sharon Mathis,"Tifton, GA",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
2,SRT Farms,"Morton, TX",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
3,Mark Duncan,"Roosevelt, UT",Failure to respond to audit (no response),2 years,11/16/2014,11/15/2016,True
4,F&W Farms,"Ingalls, KS",Failure to respond to audit (partial response),2 years,12/10/2014,12/9/2016,True
5,Mark Duncan,"Roosevelt, UT",Impeding the Audit Process – Non- Response,2 years,11/16/2014,11/15/2016,True
6,SRT Farms,"Morton, TX",Impeding the Audit Process – Non- Response,2 years,11/16/2014,11/15/2016,True
7,Annabella Land & Cattle,"Annabella, UT",Non Payment,1 year,5/9/2014,5/9/2015,False
8,"Caddo Creek Ranch, dba Paradise Ranch","Caddo, TX",Failure to respond to audit (partial response),2 years,7/20/2014,7/19/2016,False
9,Loewen Harvesting LLC,"Brownsville, TX",Failure to respond to audit (partial response),1 year,8/20/2014,8/19/2015,False


# 2. Merging and regex (17 points total)



## 2.1 Load data on job postings

The previous dataset contains a small subset of employers who faced temporary bans due to violations of H-2A program regulations

Since most of the bans have expired, we're going to see which of those employers posted new H-2A jobs in the first quarter of 2021 

Loading the `jobs_clean.csv` data stored in `pset4_inputdata`

In [14]:
jobs = pd.read_csv("pset2_inputdata/jobs.csv")
jobs.head()

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,H-300-20199-721302,Determination Issued - Withdrawn,2020-07-17 14:50:40.840,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,Y,"Fazio Farms Operating Company, LLC",NaN,...,N,1,Y,12.68,12.68,55.0,13607017661,faziofarms@gmail.com,NaN,0
1,H-300-20231-773906,Determination Issued - Certification,2020-08-20 10:38:15.620,2020-10-01 00:00:00.000,Association - Agent,N,Seasonal,N,Charlie Sunderland,Panter & Sunderland Nursery,...,N,1,N,NaN,12.68,55.0,19318083783,NaN,https://www.jobs4tn.gov/vosnet/Default.aspx,0
2,H-300-20231-774123,Determination Issued - Certification,2020-08-24 15:33:14.340,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Michael Rudebusch,NaN,...,N,1,N,NaN,12.68,55.0,19369333827,fayethlynpitre@rocketmail.com,NaN,0
3,H-300-20231-774151,Determination Issued - Certification,2020-08-21 12:08:09.760,2020-10-01 00:00:00.000,Individual Employer,N,Seasonal,N,Lodahl Farms,NaN,...,Y,2,N,NaN,12.68,55.0,14069637560,lodahl_kelsey@yahoo.com,NaN,0
4,H-300-20231-774508,Determination Issued - Certification,2020-08-20 10:17:34.530,2020-10-01 00:00:00.000,Individual Employer,Y,Seasonal,N,"Dunson Harvesting, Inc.","Dunson Harvesting, Inc.",...,Y,8,N,NaN,12.68,55.0,18632939888,NaN,www.employflorida.com,4


##  2.2 Try inner join on employer name  (2 points)

- Use the `EMPLOYER_NAME` field of the `jobs` dataset
- Use the `Name` field of the `debar_clean` dataset 

A. Use pd.merge with an inner join on those fields to see whether there are any exact matches. 

B. If there are exact matches, print the row(s) with exact matches



In [15]:
# Perform inner join on 'EMPLOYER_NAME' and 'Name' columns
inner_join_df = jobs.merge(debar_clean, left_on="EMPLOYER_NAME", right_on="Name", how="inner")

# Print the resulting DataFrame
inner_join_df


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS,Name,"City, State",Violation,Duration,Start date,End date,is_dup


## 2.3 Targeted regex (10 points total)

You want to see if you can increase the exact match rate with some basic cleaning of each 
of the employer name fields in each dataset 

### 2.3.1 Converting to upper (2 points)

A. Convert the `EMPLOYER_NAME` and `Name` fields to uppercase using list comprehension rather than df.varname.str.upper() (it's fine to do a separate list comprehension line for each of the two columns)

B. Print a random sample of 15 values of each result

C. Assign the full vector of uppercase names back to the original data, writing over the original `EMPLOYER_NAME` and `Name` columns 


In [16]:
# Convert EMPLOYER_NAME and Name fields to uppercase using list comprehension
jobs['EMPLOYER_NAME'] = [name.upper() for name in jobs['EMPLOYER_NAME']]
debar_clean['Name'] = [name.upper() for name in debar_clean['Name']]
# debar_clean['Name']
# jobs['EMPLOYER_NAME']

In [17]:
# Print a random sample of 15 values of each result
print("Random sample of EMPLOYER_NAME:")
print(jobs['EMPLOYER_NAME'].sample(15))
print("\nRandom sample of Name:")
print(debar_clean['Name'].sample(15))

Random sample of EMPLOYER_NAME:
1714                   PAUL TAYLOR
1754          HUNTERTON FARM, INC.
1743         STAN'S PURE HONEY LLC
326            DOISE CRAWFISH, LLC
335                  JEFFERY JONES
110         STRIPPIN FEATHERS, LLC
271               ALEX T DUFURRENA
2628    MONETTE FARMS ARIZONA, LLC
1761         SCHLESSIGER FARMS LLC
2101            CHAD MARTIN THEVIS
237      WESTERN RANGE ASSOCIATION
1086                 SAHA FISH PTN
1428       WALTHER FARMS, LLC - GA
582      WESTERN RANGE ASSOCIATION
1341     WESTERN RANGE ASSOCIATION
Name: EMPLOYER_NAME, dtype: object

Random sample of Name:
6                                   SRT FARMS
5                                 MARK DUNCAN
16    OLD TREE FARMS/VERPAALEN CUSTOM SERVICE
17                         CISCO PRODUCE INC.
7                     ANNABELLA LAND & CATTLE
13                      LOEWEN HARVESTING LLC
3                                 MARK DUNCAN
11                         CISCO PRODUCE INC.
12    OLD TREE F

In [18]:
# Assign the full vector of uppercase names back to the original data
jobs['EMPLOYER_NAME'] = jobs['EMPLOYER_NAME']
debar_clean['Name'] = debar_clean['Name']
# debar_clean['Name']
# jobs['EMPLOYER_NAME']

### 2.3.2 Cleaning up punctuation (4 points)

You notice that INC, CO, and LLC are sometimes followed by a period (.) but sometimes not

A. For each dataset, write a regex pattern using `re.sub` to remove the . but only if it's preceded by INC, LLC, or CO 

Make sure LLC, INC, CO remain part of the string but just without the dot

B. Test the pattern on the positive and negative example we provide below and print the result. See the Github issue for examples of what to return


**Hint**: https://stackoverflow.com/questions/7191209/python-re-sub-replace-with-matched-content



In [19]:
pos_example_1 = "CISCO PRODUCE INC."
pos_example_2 = "AVOYELLES HONEY CO., LLC"
neg_example = "E.V. RANCH LLP"

In [20]:
## insert your code here with the regex pattern for part A

## insert your code to use re.sub to apply the pattern to the test cases for part B

In [21]:
# Define the regex pattern to remove the period (.) after INC, LLC, or CO
pattern_jobs = r'\b(INC\.|LLC\.|CO\.)'

# Apply the regex pattern to remove the period (.) after INC, LLC, or CO
jobs['EMPLOYER_NAME'] = jobs['EMPLOYER_NAME'].str.replace(pattern_jobs, r'\1')

jobs['EMPLOYER_NAME']

0       FAZIO FARMS OPERATING COMPANY, LLC
1                       CHARLIE SUNDERLAND
2                        MICHAEL RUDEBUSCH
3                             LODAHL FARMS
4                  DUNSON HARVESTING, INC.
                       ...                
2715                     JAMES L SCHNELLER
2716                       STEPHEN G MYERS
2717                            ANDY POVEY
2718                SILVER CREEK SEED, LLC
2719                        HALABURA FARMS
Name: EMPLOYER_NAME, Length: 2720, dtype: object

In [22]:
# Define the regex pattern
pattern_debar = r'(INC\.|LLC\.|CO\.)'

# Apply re.sub to remove the period (.) after INC, LLC, or CO
debar_clean['Name'] = debar_clean['Name'].apply(lambda x: re.sub(pattern_debar, r'\1', x))

debar_clean['Name']

0                 ROLLO FARM LABOR CONTRACTOR
1                               SHARON MATHIS
2                                   SRT FARMS
3                                 MARK DUNCAN
4                                   F&W FARMS
5                                 MARK DUNCAN
6                                   SRT FARMS
7                     ANNABELLA LAND & CATTLE
8       CADDO CREEK RANCH, DBA PARADISE RANCH
9                       LOEWEN HARVESTING LLC
10           MAPLE RIDGE CUSTOM SERVICES, LLC
11                         CISCO PRODUCE INC.
12    OLD TREE FARMS/VERPAALEN CUSTOM SERVICE
13                      LOEWEN HARVESTING LLC
14      CADDO CREEK RANCH, DBA PARADISE RANCH
15                    ANNABELLA LAND & CATTLE
16    OLD TREE FARMS/VERPAALEN CUSTOM SERVICE
17                         CISCO PRODUCE INC.
18           MAPLE RIDGE CUSTOM SERVICES, LLC
19                               XAVIER HORNE
20                               XAVIER HORNE
Name: Name, dtype: object

In [23]:
pattern = r'(INC\.|LLC\.|CO\.)'

result_pos_1 = re.sub(pattern, r'\1', pos_example_1)
result_pos_2 = re.sub(pattern, r'\1', pos_example_2)
result_neg = re.sub(pattern, r'\1', neg_example)

# Print the results
print("Positive Example 1:", result_pos_1)
print("Positive Example 2:", result_pos_2)
print("Negative Example:", result_neg)

Positive Example 1: CISCO PRODUCE INC.
Positive Example 2: AVOYELLES HONEY CO., LLC
Negative Example: E.V. RANCH LLP


### 2.3.3 (4 points)

Use that pattern in conjunction with `re.sub` and list comprehension to clean the employer name columns in each dataset. Save the new columns as `name_clean` in each. Then, use row subsetting to (1) subset to rows that changed names and (2) for:

- `debar_clean` print the `Name` and `name_clean` columns
- `jobs` print the `EMPLOYER_NAME` and `name_clean` columns

Make sure to use the uppercase versions of the variables


In [24]:
## your code here to clean the columns

In [25]:
## your code here to print the head

## 2.4 More joins and more cleaning (5 points)

A. Conduct another inner join between `jobs` and `debar_clean` now using the `name_clean` column; print the result. Did the cleaning result in any more employers matched between the two datasets?

B. Create a new column in `debar_clean` called `name_clean_2` that uses regex to take the following name in that dataset:

- `SLASH E.V. RANCH LLP` in the `debar_clean` dataset

And cleans it up so that it matches with this employer in `jobs`

- `SLASH EV RANCH` in the `jobs` dataset

Eg a pattern to remove the dots in the EV and the space+LLP-- you can apply the pattern to all employer names in debar_clean (so don't need to worry about only applying it to that one employer)


C. Conduct a left join using `name_clean_2` as the join column where the left hand dataframe is `jobs`; right hand dataframe is `debar_clean`, store the result as a dataframe, and print the rows where the merge indicator indicates the row was found in both dataframe

**Note**: this manual cleaning process is inefficient and helps motivate why talked about fuzzy matching. Fuzzy matching could recognize that Slash EV ranch is a highly similar string to slash ev ranch llp and match them without us needing to use regex to make the strings identical.

In [26]:
## your code here

# 3. Optional extra credit 1: regex to separate companies from individuals (1 point)

You notice some employers in `debar_clean` have both the name of the company and the name of individual, e.g.:
    
COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*

Use the uppercase/cleaned `name_clean` in `debar_clean`

A. Write a regex pattern that does the following:
    - Captures the pattern that occurs before COMPANY if (COMPANY) is in string; so in example above, extracts COUNTY FAIR FARM 
    - Captures the pattern that occurs before INDIVIDUAL if (INDIVIDUAL) is also in string -- so in above, extracts ANDREW WILLIAMSON (so omit the "and")
    
B. Test the pattern on `pos_example` and `neg_example`-- make sure former returns a list (if using find.all) or match object (if using re.search) with the company name and individual name separated out; make sure latter returns empty
    
**Hints and resources**: for step A, you can either use re.search, re.match, or re.findall; don't worry about matching B&R Harvesting and Paul Cruz (Individual)

- Same regex resources as above
    

In [27]:
pos_example = "COUNTY FAIR FARM (COMPANY) AND ANDREW WILLIAMSON (INDIVIDUAL)*"
neg_example = "CISCO PRODUCE INC"

## your code here to define the pattern

## your code here to apply it to the pos_example

## your code here to apply it to the negative example

C. Iterate over the `name_clean` column in debar and use regex to create two new columns in `debar_clean`:
   - `co_name`: A column for company (full `name_clean` string if no match; pattern before COMPANY if one extracted)
   - `ind_name`: A column for individual (full `name_clean` string if no match; pattern before INDIVIDUAL if one extracted)
 


In [28]:
# your code here

   
D. Print three columns for the rows in `debar_clean` containing the negative example and positive example described above (county fair farm and cisco produce):

- `name_clean`
- `co_name`
- `ind_name`
- `Violation`

**Note**: as shown in the outcome there may be duplicates of the same company reflecting different violations

In [29]:
# your code here

# 4. Optional extra credit 2 (up to 3 points)

- For 1 point extra credit, create a visualization with 1+ of the existing fields in either the raw `jobs` or `debar` data. We'll be showing cool visualizations in class so use your imagination! Options could include visualizing between-state or over-time variation

- For 3 points extra credit instead, geocode the employer addresses in `jobs` and plot the addresses of jobs as points overlaid on top of a map of Georgia 
    - **Note**: this extra credit involves Googling since we have not yet covered spatial data. 
        - For discussion of how to geocode addresses -> lat/long, see: https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/ 
        - For discussion of plotting lat/long dots against a map, see this discussion of geopandas: https://towardsdatascience.com/plotting-maps-with-geopandas-428c97295a73
    - Relevant columns include `EMPLOYER_ADDRESS_1` 
    - The geocoding might have a long runtime so feel free to implement it in a separate .py script that you submit alongside your notebook and to just read in the geocoded data

In [30]:
## your code here